In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# **Load Dataset from Drive**

In [ ]:
source = '/content/drive/MyDrive/ktp-dataset'
#path into train and test images
train_images = os.path.join(source, 'image')
train_annotations = os.path.join(source, 'mask')

In [ ]:
sorted_image = sorted(os.listdir(train_images))
sorted_mask = sorted(os.listdir(train_annotations))

In [ ]:
img_size = 512

In [ ]:
def get_segmented_img(img):
  
    labels=np.zeros((img_size,img_size,1))
    img=cv2.resize(img,(img_size,img_size))
    img=img[:,:,0]
    labels[:,:,0]=(img!=0).astype(int)


    return labels

In [ ]:
x = []
y = []

for index in range (len(os.listdir(train_images))):
    img = cv2.imread(os.path.join(train_images, sorted_image[index]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,img=cv2.threshold(img,150,255,cv2.THRESH_BINARY_INV)
    img=cv2.resize(img,(img_size,img_size))
    img=np.expand_dims(img,axis=-1)
    img=img/255
    
    annotation = cv2.imread(os.path.join(train_annotations, sorted_mask[index]))
    annotation = cv2.cvtColor(annotation, cv2.COLOR_BGR2GRAY)
    annotation=cv2.resize(annotation,(img_size,img_size))
    annotation = np.stack((annotation,)*3, axis=-1)
    annotation =get_segmented_img(annotation)
    
    x.append(img)
    y.append(annotation)

In [ ]:
# Split Dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 1)

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print(x_train.shape)
print(x_test.shape)

(273, 512, 512, 1)
(118, 512, 512, 1)


# **U-Net Segmentation using CNN model**

In [ ]:
img_size = 512
inputs = Input((img_size, img_size, 1))

conv1 = Conv2D(64, 3, activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
pool1 = MaxPooling2D(2, 2)(conv1)

conv2 = Conv2D(128, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
pool2 = MaxPooling2D(2, 2)(conv2)

conv3 = Conv2D(256, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
pool3 = MaxPooling2D(2, 2)(conv3)

conv4 = Conv2D(512, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
pool4 = MaxPooling2D(2, 2)(conv4)

conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
dropout5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation='relu', kernel_initializer='he_normal', padding='same')(UpSampling2D(size=(2, 2))(dropout5))
merge6 = concatenate([conv4, up6], axis=3)
conv6 = Conv2D(512, 3, activation='relu', kernel_initializer='he_normal', padding='same')(merge6)
conv6 = Conv2D(512, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

up7 = Conv2D(256, 2, activation='relu', kernel_initializer='he_normal', padding='same')(UpSampling2D(size=(2, 2))(conv6))
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(256, 3, activation='relu', kernel_initializer='he_normal', padding='same')(merge7)
conv7 = Conv2D(256, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv7)

up8 = Conv2D(128, 2, activation='relu', kernel_initializer='he_normal', padding='same')(UpSampling2D(size=(2, 2))(conv7))
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(128, 3, activation='relu', kernel_initializer='he_normal', padding='same')(merge8)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

up9 = Conv2D(64, 2, activation='relu', kernel_initializer='he_normal', padding='same')(UpSampling2D(size=(2, 2))(conv8))
merge9 = concatenate([conv1, up9], axis=3)
conv9 = Conv2D(64, 3, activation='relu', kernel_initializer='he_normal', padding='same')(merge9)
conv9 = Conv2D(64, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv9)

conv9 = Conv2D(2, 3, activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

model = Model(inputs=inputs, outputs=conv10)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 512, 512, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 512, 512, 64  36928       ['conv2d_24[0][0]']              
                                )                                                             

# **Train the model**

In [ ]:
model.fit(x_train,y_train,
                    epochs=6,
                    steps_per_epoch= 500,
                    validation_data=(x_test, y_test),
                    validation_steps= 400,
                    shuffle=1)

In [ ]:
model.save('/content/drive/MyDrive/model/bounding_ktp01.h5')

In [ ]:
model = load_model('/content/drive/MyDrive/model/bounding_ktp01.h5')

# **Predict**

In [ ]:
#predict

file_test ='/content/ktp01.jpg'
img=cv2.imread(file_test,0)
ret,img=cv2.threshold(img,150,255,cv2.THRESH_BINARY_INV)
img=cv2.resize(img,(512,512))
img=np.expand_dims(img,axis=-1)
img=img/255

img=np.expand_dims(img,axis=0)
pred=model.predict(img)
pred=np.squeeze(np.squeeze(pred,axis=0),axis=-1)
plt.imshow(pred,cmap='gray')
plt.imsave('/content/test_img_mask.png',pred)

img = cv2.imread('/content/test_img_mask.png',0) 
print(img.shape)
cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU,img)
ori_img = cv2.imread(file_test)
ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
ori_img = cv2.resize(ori_img, (512,512))

roi_img = []

roi_number = 0
contours, hier = cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
contours = sorted(contours, key = lambda x: cv2.boundingRect(x)[0] + cv2.boundingRect(x)[1] * img.shape[1])
for c in contours:
    # bounding rect
    x, y, w, h = cv2.boundingRect(c)
    if w > 40:
    # create a rectangle to visualize the bounding rect
      cv2.rectangle(ori_img, (x, y), (x + w, y + h), (36,255,12), 2)
      ROI = ori_img[y:y+h, x:x+w]
      roi_img.append(ROI)
      # cv2.imwrite('/content/test_{}.png'.format(roi_number), ROI)
      roi_number += 1

    #cv2.drawContours(img, contours, -1, (255, 255, 0), 1)

plt.imshow(ori_img)
plt.show()
     